In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

def generate_summary(input_text):
    # Load the Pegasus Large model and tokenizer
    model_name = 'google/pegasus-large'
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)

    # Preprocess the text
    inputs = tokenizer(input_text, truncation=True, padding='longest', max_length=1024, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Generate summary
    summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_beams=4, length_penalty=2.0)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [24]:
# User-facing Jupyter Notebook frontend using ipywidgets
import ipywidgets as widgets
from IPython.display import display, Javascript
from ipywidgets import HBox

# Create input and output widgets
input_text_widget = widgets.Textarea(
    placeholder='Enter your text here...',
    layout=widgets.Layout(width='500px', height='200px')
)
output_text_widget = widgets.Textarea(
    placeholder='Summary will be displayed here...',
    layout=widgets.Layout(width='500px', height='200px'),
    disabled=True
)

# Create reset button
reset_button = widgets.Button(description='Reset', button_style='danger')
reset_button.layout.margin = '10px'

def reset_inputs(button):
    input_text_widget.value = ''
    output_text_widget.value = ''
    copy_button.disabled = True

# Attach reset event handler to the button
reset_button.on_click(reset_inputs)

# Create button and define click event handler
button = widgets.Button(description='Generate Summary')
button.layout.margin = '10px'

def generate_summary_on_click(button):
    input_text = input_text_widget.value.strip()
    if input_text:
        # Disable the button and update the text
        button.disabled = True
        button.description = 'Generating Summary...'

        summary = generate_summary(input_text)
        output_text_widget.value = summary

        # Re-enable the button and reset the text
        button.disabled = False
        button.description = 'Generate Summary'
        copy_button.disabled = False if summary else True

# Attach click event handler to button
button.on_click(generate_summary_on_click)

# Display widgets
buttons_box = HBox([button, reset_button])
display(input_text_widget, buttons_box, output_text_widget)


Textarea(value='', layout=Layout(height='200px', width='500px'), placeholder='Enter your text here...')

Textarea(value='', disabled=True, layout=Layout(height='200px', width='500px'), placeholder='Summary will be d…